In [1]:
import sklearn
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('figure', figsize=[10,5])
import mlrose_hiive
from sklearn.metrics import f1_score
from functools import partial

In [2]:
np.random.seed(44)
wine_dataset = pd.read_csv("wine_dataset.csv")
print(wine_dataset.info())
wine_dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
class                      5000 non-null int64
Alcohol                    5000 non-null float64
Malic_acid                 5000 non-null float64
Ash                        5000 non-null float64
Alcalinity_of_ash          5000 non-null float64
Magnesium                  5000 non-null float64
Total_phenols              5000 non-null float64
Flavanoids                 5000 non-null float64
Nonflavanoid_phenols       5000 non-null float64
Proanthocyanins            5000 non-null float64
Color_intensity            5000 non-null float64
Hue                        5000 non-null float64
Od_Hod_of_diluted_wines    5000 non-null float64
Proline                    5000 non-null float64
dtypes: float64(13), int64(1)
memory usage: 547.0 KB
None


,class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,Od_Hod_of_diluted_wines,Proline
0,1,13.761443,0.972960,2.191146,14.349954,106.525734,2.288580,2.584551,0.418044,1.862804,3.086808,1.169861,2.819086,850.678431
1,2,11.897923,1.297780,2.199666,17.478054,104.205421,2.716657,1.855066,0.318077,1.029640,9.103668,1.353718,2.780333,332.922006
2,3,12.659632,1.748552,2.438597,21.584696,88.079451,1.919476,0.681341,0.538078,1.327974,8.803633,0.643678,1.569951,633.531687
3,1,13.054828,1.817559,1.893926,15.390157,108.589893,2.662509,2.815723,0.278535,1.473812,6.386732,0.820398,3.694865,880.103325
4,3,11.769431,3.017528,3.013448,17.081528,86.525235,2.224963,0.846229,0.286065,1.644007,10.383464,0.734288,2.747013,475.152492


In [3]:
# normalize features for better performance 
from sklearn import preprocessing
target = wine_dataset["class"]
x = wine_dataset.values # convert to numpy array
print(x.shape)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
print(x_scaled.shape)
wine_dataset = pd.DataFrame(x_scaled, columns=wine_dataset.columns)
wine_dataset["class"] = target
wine_dataset.head()

(5000, 14)
(5000, 14)


,class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,Od_Hod_of_diluted_wines,Proline
0,1,0.726791,0.085805,0.432125,0.200468,0.392663,0.452788,0.559874,0.477090,0.489749,0.163244,0.635628,0.584439,0.399009
1,2,0.237919,0.142156,0.437079,0.357903,0.368467,0.599717,0.382811,0.322602,0.258369,0.665030,0.790293,0.571170,0.065151
2,3,0.437744,0.220358,0.576004,0.564587,0.200303,0.326101,0.097921,0.662590,0.341220,0.640008,0.192992,0.156748,0.258989
3,1,0.541419,0.232330,0.259309,0.252821,0.414189,0.581132,0.615985,0.261494,0.381721,0.438447,0.341653,0.884296,0.417983
4,3,0.204211,0.440507,0.910246,0.337946,0.184095,0.430953,0.137943,0.273130,0.428986,0.771760,0.269215,0.559762,0.156864


In [4]:
# stratified sampling
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1 , test_size=0.2, random_state=30)
for train_ind, test_ind in split.split(wine_dataset, wine_dataset["class"]):
    strat_train_set = wine_dataset.loc[train_ind]
    strat_test_set = wine_dataset.loc[test_ind]

In [5]:
train_set = strat_train_set
test_set = strat_test_set
# stratified sampling is not needed
train_y = train_set[["class"]]
train_X = train_set.drop("class", axis=1)
test_y = test_set[["class"]]
test_X = test_set.drop("class", axis=1)
train_set.info(), test_set.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 1725 to 3709
Data columns (total 14 columns):
class                      4000 non-null int64
Alcohol                    4000 non-null float64
Malic_acid                 4000 non-null float64
Ash                        4000 non-null float64
Alcalinity_of_ash          4000 non-null float64
Magnesium                  4000 non-null float64
Total_phenols              4000 non-null float64
Flavanoids                 4000 non-null float64
Nonflavanoid_phenols       4000 non-null float64
Proanthocyanins            4000 non-null float64
Color_intensity            4000 non-null float64
Hue                        4000 non-null float64
Od_Hod_of_diluted_wines    4000 non-null float64
Proline                    4000 non-null float64
dtypes: float64(13), int64(1)
memory usage: 468.8 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 387 to 4415
Data columns (total 14 columns):
class                      1000 non-null int

(None, None)

In [8]:
%%time
f1_labs = partial(f1_score, average="weighted")
grid_search_parameters = ({
  'learning_rate': [0.01, 0.001],
  'restarts': [5, 25, 50],
  'activation': [mlrose_hiive.neural.activation.relu]
})
rhc_nn = mlrose_hiive.NNGSRunner(train_X, pd.get_dummies(train_y.values.ravel()).values, 
                                 test_X, pd.get_dummies(test_y.values.ravel()).values, 
                                 "NeuralNetworkRHC",
                                 output_directory="./",
                                 seed=10, iteration_list=[10000], 
                                 algorithm=mlrose_hiive.random_hill_climb,
                                 hidden_layer_sizes=[[60,60]],
                                 grid_search_parameters=grid_search_parameters,
                                 grid_search_scorer_method=f1_labs,
                                 n_jobs=-2, cv=5)
result = rhc_nn.run()

Running nngs_rhc
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  30 out of  30 | elapsed: 52.7min finished


*****************
*** Run START ***
*****************
max_iters:[100], learning_rate:[0.01], restarts:[50], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.15103284  0.16559681
  0.72446653]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[0]
runner_name:[nngs_rhc], experiment_name:[NeuralNetworkRHC], attempt:[0], iteration:[0], done:[False], time:[0.06], fitness:[4.0498]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.15103284  0.16559681//  0.72446653]...

max_iters:[100], learning_rate:[0.01], restarts:[50], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.15103284  0.16559681
  0.72446653]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[0]
runner_name:[nngs_rhc], experiment_name:[Neur

In [21]:
result[2]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_learning_rate,param_restarts,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,51.444553,1.009739,0.021213,0.007156,relu,"[60, 60]",0.010,5,{'activation': <function relu at 0x7f9c85a9329...,0.228489,0.240795,0.249942,0.239708,0.26005,0.243797,0.010602,5,0.247954,0.244763,0.242457,0.244920,0.239860,0.243991,0.002705
1,474.523966,6.637853,0.022352,0.010333,relu,"[60, 60]",0.010,25,{'activation': <function relu at 0x7f9c85a9329...,0.231804,0.252091,0.255778,0.241000,0.25384,0.246903,0.009128,3,0.250895,0.245756,0.244806,0.248339,0.245243,0.247008,0.002298
2,1536.826047,11.556979,0.030121,0.017126,relu,"[60, 60]",0.010,50,{'activation': <function relu at 0x7f9c85a9329...,0.229942,0.248157,0.258725,0.241186,0.26390,0.248382,0.012163,1,0.253376,0.248660,0.246007,0.250354,0.244645,0.248608,0.003107
3,51.455579,1.333915,0.031811,0.006626,relu,"[60, 60]",0.001,5,{'activation': <function relu at 0x7f9c85a9329...,0.228489,0.240795,0.249942,0.239708,0.26005,0.243797,0.010602,5,0.247954,0.244763,0.242457,0.244920,0.239860,0.243991,0.002705
4,467.678722,11.246476,0.023133,0.007870,relu,"[60, 60]",0.001,25,{'activation': <function relu at 0x7f9c85a9329...,0.231804,0.252091,0.255778,0.241000,0.25384,0.246903,0.009128,3,0.250895,0.245756,0.244806,0.248339,0.245243,0.247008,0.002298
5,1258.814184,75.378280,0.026075,0.015492,relu,"[60, 60]",0.001,50,{'activation': <function relu at 0x7f9c85a9329...,0.229942,0.248157,0.258725,0.241186,0.26390,0.248382,0.012163,1,0.253376,0.248660,0.246007,0.250354,0.244645,0.248608,0.003107


In [18]:
from sklearn.metrics import f1_score, accuracy_score, classification_report
y_pred = result[3].predict(test_X)
print(classification_report(pd.get_dummies(test_y.values.ravel()).values, y_pred))

              precision    recall  f1-score   support

           0       0.34      0.98      0.51       315
           1       0.86      0.01      0.03       403
           2       0.73      0.21      0.33       282

   micro avg       0.38      0.38      0.38      1000
   macro avg       0.64      0.40      0.29      1000
weighted avg       0.66      0.38      0.26      1000
 samples avg       0.38      0.38      0.38      1000



In [19]:
y_pred_train = result[3].predict(train_X)
print(classification_report(pd.get_dummies(train_y.values.ravel()).values, y_pred_train))

              precision    recall  f1-score   support

           0       0.34      0.98      0.51      1262
           1       0.81      0.01      0.02      1610
           2       0.58      0.19      0.28      1128

   micro avg       0.37      0.37      0.37      4000
   macro avg       0.58      0.39      0.27      4000
weighted avg       0.60      0.37      0.25      4000
 samples avg       0.37      0.37      0.37      4000

